In [1]:
import os
import librosa
import numpy as np
from random import choice
import soundfile as sf

# Ruta de las carpetas originales y destino
original_folders = ["crackle", "wheeze", "crackle_wheeze"]
output_folders = ["crackle", "wheeze", "crackle_wheeze"]

# Definir las técnicas de augmentación
def pitch_shift(audio, sr):
    semi_tones = np.random.uniform(-2, 2)  # Cambio de tono entre -2 y 2 semitonos
    return librosa.effects.pitch_shift(audio, sr=sr, n_steps=semi_tones)

def random_shift(audio):
    shift_range = np.random.randint(-1000, 1000)  # Desplazamiento aleatorio entre -1000 y 1000 muestras
    return np.roll(audio, shift_range)

def volume_adjust(audio):
    factor = np.random.uniform(0.5, 1.5)  # Factor de ajuste de volumen entre 0.5 y 1.5
    return audio * factor

def time_stretch(audio):
    stretch_factor = np.random.uniform(0.8, 1.2)  # Factor de estiramiento temporal entre 0.8 y 1.2
    return librosa.effects.time_stretch(audio, rate=stretch_factor)

# Mapeo de funciones de augmentación a nombres de archivos
augmentation_functions = {
    "pitch_shift": pitch_shift,
    "random_shift": random_shift,
    "volume_adjust": volume_adjust,
    "time_stretch": time_stretch
}

# Función para aplicar augmentación y guardar archivos
def augment_and_save(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".wav"):
            file_path = os.path.join(input_folder, filename)
            audio, sr = librosa.load(file_path, sr=None)

            # Seleccionar una técnica de augmentación aleatoria
            augmentation_name = choice(list(augmentation_functions.keys()))

            if augmentation_name == "pitch_shift":
                augmented_audio = augmentation_functions[augmentation_name](audio, sr)
            else:
                augmented_audio = augmentation_functions[augmentation_name](audio)

            # Guardar el archivo original y el aumentado
            output_filename = f"{augmentation_name}_{filename}"
            output_file_path = os.path.join(output_folder, output_filename)
            sf.write(output_file_path, augmented_audio, sr)

            # Guardar el archivo original
            original_output_file_path = os.path.join(output_folder, filename)
            sf.write(original_output_file_path, audio, sr)

# Aplicar augmentación y guardar archivos para las carpetas seleccionadas
for original_folder, output_folder in zip(original_folders, output_folders):
    input_folder_path = os.path.join("./AudiosOfficial/train", original_folder)
    output_folder_path = os.path.join("./AudiosOfficialAUG/train", output_folder)
    augment_and_save(input_folder_path, output_folder_path)